<a href="https://colab.research.google.com/github/momizi06/AnythingV3.0_Colab/blob/main/SDWebUI_V2_faster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi
!nvcc -V
!free -h

Sat Feb 11 01:42:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0    29W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%cd /content
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui

%cd stable-diffusion-webui
!git pull

# Install image browser extension
!git clone https://github.com/yfszzx/stable-diffusion-webui-images-browser extensions/images-browser

!rm -r /content/sample_data
!apt install -y -qq aria2

# To Skip install requirements if WebUI is crashed and re-run.
run_counter = 0


/content
Cloning into 'stable-diffusion-webui'...
remote: Enumerating objects: 16079, done.
remote: Total 16079 (delta 0), reused 0 (delta 0), pack-reused 16079
Receiving objects: 100% (16079/16079), 27.23 MiB | 39.16 MiB/s, done.
Resolving deltas: 100% (11293/11293), done.
/content/stable-diffusion-webui
Already up to date.
Cloning into 'extensions/images-browser'...
remote: Enumerating objects: 143, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 143 (delta 25), reused 22 (delta 22), pack-reused 109
Receiving objects: 100% (143/143), 37.96 KiB | 3.45 MiB/s, done.
Resolving deltas: 100% (51/51), done.
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to

In [4]:
%cd /content/stable-diffusion-webui/models/Stable-diffusion/

Model = "AnythingV3.0" #@param ["AnythingV3.0", "AnythingV4.0", "AnythingV4.5", "TrinArt Derrida model (Characters v2)", "ACertainThing", "8528d-final", "Counterfeit-V2.0", "Counterfeit-V2.1", "Counterfeit-V2.2", "Counterfeit-V2.5", "AbyssOrangeMix2_nsfw"]

def FileNamePredict(url: str):
    import re  
    temp = re.search(r'([^/]*?)$', url)  
    PredictedName = temp.group()  
    return PredictedName  
    

def CheckEmpty(text: str):
    if (len(text) == 0):
        return True
    else:
        return False



def ModelDL(modelURL: str, modelName: str, **kargs):
    if CheckEmpty(modelURL):
        print('Please input model URL.')
        return
    if CheckEmpty(modelName):
        modelName = FileNamePredict(modelURL)
        print(f"model name has predicted as '{modelName}'.")
    print(f"Install \033[32;1m{modelName}\033[0m ({Model}).")
    !aria2c --summary-interval=10 -x 3 --allow-overwrite=false -o {modelName} \
        {modelURL}
    return modelName
    

def vaeDL(vaeURL: str, vaeName: str, **kargs):
    if CheckEmpty(vaeURL):
        print('Please input VAE URL. VAE install is skipped.')
        return "none"
    if (vaeURL == "none"):
        return "none"
    if CheckEmpty(vaeName):
        vaeName = FileNamePredict(vaeURL)
        print(f"VAE name has predicted as '{vaeName}'.")
    !curl -Lo /content/stable-diffusion-webui/models/VAE/{vaeName} {vaeURL}
    return vaeName


def SetStartUpModel(modelName: str, vaeName: str):
    ckpt_option=f"--ckpt /content/stable-diffusion-webui/models/Stable-diffusion/{modelName}"
    vae_option=f"--vae-path /content/stable-diffusion-webui/models/VAE/{vaeName}"
    if (vaeName == "none"):
        vae_option=""
    global ModelOption
    ModelOption = f"{ckpt_option} {vae_option}"
    return 
    



Unko = {
    'AnythingV3.0': {
        'modelURL': 'https://huggingface.co/ckpt/anything-v3.0/resolve/main/Anything-V3.0-pruned.safetensors',
        'vaeURL': 'https://huggingface.co/ckpt/anything-v3.0/resolve/main/Anything-V3.0.vae.pt',
        'modelName': 'Anything-V3.0-pruned.safetensors',
        'vaeName': 'Anything-V3.0.vae.pt'
        },
        
    'AnythingV4.0': {
        'modelURL': 'https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.0-pruned.safetensors',
        'vaeURL': 'https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.0.vae.pt',
        'modelName': 'Anything-v4.0-pruned.safetensors',
        'vaeName': 'Anything-v4.0.vae.pt'
        },
        
    'AnythingV4.5': {
        'modelURL': 'https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.5-pruned.safetensors',
        'vaeURL': 'https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.0.vae.pt',
        'modelName': 'Anything-v4.5-pruned.safetensors',
        'vaeName': 'Anything-v4.0.vae.pt'
        },
    
    'TrinArt Derrida model (Characters v2)': {
        'modelURL': 'https://huggingface.co/naclbit/trinart_derrida_characters_v2_stable_diffusion/resolve/main/derrida_final.ckpt',
        'vaeURL': 'https://huggingface.co/naclbit/trinart_derrida_characters_v2_stable_diffusion/resolve/main/autoencoder_fix_kl-f8-trinart_characters.ckpt',
        'modelName': 'derrida_final.ckpt',
        'vaeName': 'TrinArt-V2.vae.pt'
        },
    
    'ACertainThing': {
        'modelURL': 'https://huggingface.co/JosephusCheung/ACertainThing/resolve/main/ACertainThing-half.ckpt',
        'vaeURL': 'none',
        'modelName': 'ACertainThing-half.ckpt',
        'vaeName': ''
        },
    
    '8528d-final': {
        'modelURL': 'https://huggingface.co/852wa/8528-diffusion/resolve/main/8528d-final.ckpt',
        'vaeURL': 'none',
        'modelName': '8528d-final.ckpt',
        'vaeName': ''
        },
    
    'Counterfeit-V2.0': {
        'modelURL': 'https://huggingface.co/gsdf/Counterfeit-V2.0/resolve/main/Counterfeit-V2.0fp16.safetensors',
        'vaeURL': 'none',
        'modelName': 'Counterfeit-V2.0fp16.safetensors',
        'vaeName': ''
        },
        
    'Counterfeit-V2.1': {
        'modelURL': 'https://huggingface.co/gsdf/Counterfeit-V2.5/resolve/main/Counterfeit-V2.1.safetensors',
        'vaeURL': 'none',
        'modelName': 'Counterfeit-V2.1.safetensors',
        'vaeName': ''
        },
        
    'Counterfeit-V2.2': {
        'modelURL': 'https://huggingface.co/gsdf/Counterfeit-V2.5/resolve/main/Counterfeit-V2.2.safetensors',
        'vaeURL': 'none',
        'modelName': 'Counterfeit-V2.2.safetensors',
        'vaeName': ''
        },
        
    'Counterfeit-V2.5': {
        'modelURL': 'https://huggingface.co/gsdf/Counterfeit-V2.5/resolve/main/Counterfeit-V2.5_pruned.safetensors',
        'vaeURL': 'https://huggingface.co/gsdf/Counterfeit-V2.5/resolve/main/Counterfeit-V2.5.vae.pt',
        'modelName': 'Counterfeit-V2.5_pruned.safetensors',
        'vaeName': 'Counterfeit-V2.5.vae.pt'
        },
        
    'AbyssOrangeMix2_nsfw': {
        'modelURL': 'https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_nsfw.safetensors',
        'vaeURL': 'https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt',
        'modelName': 'AbyssOrangeMix2_nsfw.safetensors',
        'vaeName': 'orangemix.vae.pt'
        },
}


url_option = "--share "
  

no_half_vae = False #@param {type: "boolean"}
if (no_half_vae == True):
    ckpt_option="--no-half-vae "+ckpt_option
    print("use '--no-half-vae'")


install_EasyNegative = True #@param {type: "boolean"}
if (install_EasyNegative == True):
    print('Install EasyNegative')
    !curl -Lo /content/stable-diffusion-webui/embeddings/EasyNegative.safetensors https://huggingface.co/datasets/gsdf/EasyNegative/resolve/main/EasyNegative.safetensors



ModelDict = Unko[Model]

SetStartUpModel(ModelDL(**ModelDict),vaeDL(**ModelDict))


print(f"""SetUp completed.
Setting List
    Model: {Model}
    Use '--no-half-vae': {no_half_vae}
    Install EasyNegative: {install_EasyNegative}
    ModelOption: {ModelOption}
""")


/content/stable-diffusion-webui/models/Stable-diffusion
Install EasyNegative
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1145  100  1145    0     0  11225      0 --:--:-- --:--:-- --:--:-- 11336
100 24655  100 24655    0     0  99016      0 --:--:-- --:--:-- --:--:--  213k
Install Anything-V3.0-pruned.safetensors (AnythingV3.0).

02/11 01:44:35 [NOTICE] Downloading 1 item(s)

02/11 01:44:35 [NOTICE] CUID#7 - Redirecting to https://cdn-lfs.huggingface.co/repos/43/3b/433bc34278d232abc26eacd1755075b8526959b10c4fa7332f4269bf9e9b4b6e/5ba5b79fd1006f4469857f09b62bf552d3da038974e9337b2b1594b800444bf2?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27Anything-V3.0-pruned.safetensors%3B+filename%3D%22Anything-V3.0-pruned.safetensors%22%3B&Expires=1676339076&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzQzLzNiLzQzM2Jj

In [5]:
%cd /content/stable-diffusion-webui
if (run_counter == 0):
    # hacks to load model in colab
    !sed -i 's/map_location="cpu"/map_location="cuda"/g' /content/stable-diffusion-webui/modules/sd_models.py
    !pip install -qq fastapi==0.90.1
    !pip install -U -qq pip
    !pip install -qq -r requirements.txt
    !echo "" > /tmp/requirements_empty.txt
    run_counter = 1

normal_args="--enable-console-prompts --gradio-auth unko:gabaana"
!COMMANDLINE_ARGS="{url_option} {normal_args} {ModelOption}" REQS_FILE="/tmp/requirements_empty.txt" python launch.py

/content/stable-diffusion-webui
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 KB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 25.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.9 MB/s eta 0:00:00
     

In [6]:
!zip -r /content/stable-diffusion-webui /content/stable-diffusion-webui/outputs

direct_dl = True #@param {type: "boolean"}
if (direct_dl == True):
    from google.colab import files
    files.download('/content/stable-diffusion-webui.zip')

  adding: content/stable-diffusion-webui/outputs/ (stored 0%)
  adding: content/stable-diffusion-webui/outputs/extras-images/ (stored 0%)
  adding: content/stable-diffusion-webui/outputs/img2img-grids/ (stored 0%)
  adding: content/stable-diffusion-webui/outputs/img2img-images/ (stored 0%)
  adding: content/stable-diffusion-webui/outputs/txt2img-grids/ (stored 0%)
  adding: content/stable-diffusion-webui/outputs/txt2img-images/ (stored 0%)
  adding: content/stable-diffusion-webui/outputs/txt2img-images/2023-02-11/ (stored 0%)
  adding: content/stable-diffusion-webui/outputs/txt2img-images/2023-02-11/00000-1046168986.png (deflated 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
%cd /content

SafetyLock = True #@param {type: "boolean"}
if (SafetyLock == False):
    !rm -r stable-diffusion-webui/outputs/
    print('completed.')
else:
    print('SafetyLock is Enable. To delete all outputs, please disable SafetyLock')

/content


In [ ]:
#@title save outputs to Google Drive
%cd /content/
import datetime

dt_now = datetime.datetime.now()
FormattedTime = dt_now.strftime('%Y-%m-%d_%H:%M:%S')
output_name_type = "date" #@param ['date', 'custom']

def SetOutputsFolderName(name: str):
    global FolderName
    FolderName = name+'_outputs'
    return

if (output_name_type == 'custom'):
    custom_name = "" #@param {type: 'string'}
    SetOutputsFolderName(custom_name)
else:
    SetOutputsFolderName(FormattedTime)
    
%cp -r "/content/stable-diffusion-webui/outputs" "/content"
%mv -T "/content/outputs" $FolderName

from google.colab import drive # type: ignore
try:
    drive_path = "/content/drive/"
    drive.mount(drive_path,force_remount=False)
except:
    print("...error mounting drive or with drive path variables")
    
!cp -r /content/$FolderName "/content/drive/MyDrive"


In [ ]:
!apt install -y screenfetch
!screenfetch

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  bc giblib1 libid3tag0 libimlib2 libpci3 pci.ids pciutils scrot
The following NEW packages will be installed:
  bc giblib1 libid3tag0 libimlib2 libpci3 pci.ids pciutils screenfetch scrot
0 upgraded, 9 newly installed, 0 to remove and 21 not upgraded.
Need to get 689 kB of archives.
After this operation, 3,020 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 pci.ids all 0.0~2020.03.20-1 [217 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libpci3 amd64 1:3.6.4-1ubuntu0.20.04.1 [26.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 pciutils amd64 1:3.6.4-1ubuntu0.20.04.1 [56.5 kB]
Get:4 http://archive.ubuntu.com/ubuntu fo